In [113]:
import pandas as pd
from sqlite3 import connect
import plotly.express as px

# Update data

In [1]:
data_url = r'https://covid.ourworldindata.org/data/full_data.csv'

In [114]:
source_data = pd.read_csv(data_url)
with connect("covid-19.db") as conn:
  source_data.to_sql('source_data',conn,if_exists="replace")

# Plot Data

In [105]:
def plot_cases(starting_threshold=100):
    sql = '''select source_data.location, source_data.date, source_data.total_cases, source_data.new_cases,
      julianday(source_data.date) - julianday(day_n.date) as day 
      from source_data left join (select location, min(date) as date from source_data where total_cases > ? GROUP by location) as day_n
      on source_data.location = day_n.location 
      where day is not NULL and day >= 0 and source_data.location != "International"'''
    with connect("covid-19.db") as conn:
        px.line(pd.read_sql(sql,conn,params=[starting_threshold]),
                x='day',y='total_cases',
                hover_name='location',
                hover_data=['total_cases','new_cases'],
                color='location'
               ).write_html(f"plot-{starting_threshold}.html",auto_open=True)

In [112]:
plot_cases(150)